## Introduction
In this article we will discuss a very common *decision problem*: how best to travel from a starting point `A` to some target destination `Z` in the *"best"* manner possible. Of course, as we previously discussed in the [last article](https://diogenesanalytics.com/blog/2025/02/26/reasoning-about-utility), it is the *utility* that must be defined in order to *quantitatively* define what is best. The majority of the article will present the *mathematical* definition of the utility function $U(x)$ for the *travel problem*, as well as the reasoning behind the definition, and finally a simple application.

## Utility Function Defined
Before we really get into explaining the *reasoning* behind our choice of *function definition*, we will first simply show the mathematical formula in all its glory:

$$
U(x) = \frac{D_{\text{ideal}}}{M \cdot T \cdot S \cdot D_{\text{actual}}}
$$

Where:

- $D_{\text{ideal}}$ is the ideal (straight-line) distance between the start and end points of the journey.
- $D_{\text{actual}}$ is the actual traveled distance.
- $M$ is the monetary cost.
- $T$ is the total time.
- $S$ is the stress.

In the next section we will explore the *"reasoning"* behind this equation.

## Reasoning About Travel Utility
To understand how we arrived at such an equation, we must ask ourselves a simple question: what is the purpose of travel? Simply put, *travel* is about getting from one point to another. That could be called the *"reward"* or *"benefit."* But the cost of said travel does not include just money $M$, but also time $T$, and even something more abstract, which is stress $S$.
So we can think about the *"utility"* of such a problem as simply:

$$
U(x) = \frac{\text{What You Want}}{\text{What It Costs}}
$$

In this case we want to travel some distance $D$ between two points, but we will be forced to pay some cost term that includes *money* $M$, *time* $T$, and stress $S$:

$$
U(x) = \frac{D}{M \cdot T \cdot S}
$$

## Distance Efficiency
But there is something else to consider: should utility increase or decrease as we are *"forced"* to travel **more** distance than the straight line path?

In [ ]:
# libs for downloading earth map data
import pathlib
import tempfile
import urllib.request
import zipfile
from tqdm import tqdm

# natural earth 10m cultural shape file
shape_file_url = (
    "https://naciscdn.org/naturalearth/10m/cultural/10m_cultural.zip"
)

# directory to save the downloaded file
data_dir = pathlib.Path("./data/geo/")

# directory to extract the contents of the ZIP file
extract_dir = data_dir / "natural_earth_10m_cultural"

def download_with_progress(url: str, filename: pathlib.Path) -> None:
    """Utility function to download files with progress bar."""
    # set headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # create request
    request = urllib.request.Request(url, headers=headers)

    # open URL
    response = urllib.request.urlopen(request)

    # get total file size
    total = int(response.getheader('Content-Length', 0))

    # set block size for updating
    block_size = 1024

    # create progress bar
    tqdm_bar = tqdm(total=total, unit='B', unit_scale=True, desc=filename.name)

    # open new file
    with open(filename, 'wb') as file:
        # loop over blocks
        while True:
            # get next data chunk
            buffer = response.read(block_size)

            # quit if finished
            if not buffer:
                break

            # write out data chunk
            file.write(buffer)

            # update progress bar
            tqdm_bar.update(len(buffer))

    # finished
    tqdm_bar.close()

# check for previous download
if not extract_dir.exists():
    # create the parent tree (in case it doesn't exist)
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # create a temporary directory to download the file
    with tempfile.TemporaryDirectory() as temp_dir:
        # filename for the downloaded ZIP file
        zip_filename = pathlib.Path(temp_dir) / "10m_cultural.zip"
        
        # download the ZIP file with progress bar
        download_with_progress(shape_file_url, zip_filename)
    
        # extract the contents of the ZIP file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

In [ ]:
import geopandas as gpd

# get world geo data
world_map = gpd.read_file(extract_dir / "10m_cultural")

# filter for South American countries
south_america = world_map[(world_map["CONTINENT"] == "South America")]

In [ ]:
# define all reasonably significant airports in South America, sorted by country
airports_code_map = {

    # Argentina
    "EZE": {"city": "Buenos Aires", "country": "Argentina", "coords": (-58.3816, -34.6037)},
    "COR": {"city": "Cordoba", "country": "Argentina", "coords": (-64.1888, -31.4201)},
    "ROS": {"city": "Rosario", "country": "Argentina", "coords": (-60.6393, -32.9468)},
    "MDZ": {"city": "Mendoza", "country": "Argentina", "coords": (-68.8458, -32.8908)},
    "MDQ": {"city": "Mar del Plata", "country": "Argentina", "coords": (-57.5426, -38.0055)},
    "NQN": {"city": "Neuquén", "country": "Argentina", "coords": (-68.0578, -38.9516)},
    
    # Bolivia
    "VVI": {"city": "Santa Cruz", "country": "Bolivia", "coords": (-63.1805, -17.7892)},
    "LPB": {"city": "La Paz", "country": "Bolivia", "coords": (-68.1193, -16.5000)},
    
    # Brazil
    "GRU": {"city": "São Paulo", "country": "Brazil", "coords": (-46.6333, -23.5505)},
    "GIG": {"city": "Rio de Janeiro", "country": "Brazil", "coords": (-43.1729, -22.9068)},
    "BSB": {"city": "Brasília", "country": "Brazil", "coords": (-47.9292, -15.7801)},
    "MAO": {"city": "Manaus", "country": "Brazil", "coords": (-60.0217, -3.1019)},
    "FOR": {"city": "Fortaleza", "country": "Brazil", "coords": (-38.5267, -3.7172)},
    "REC": {"city": "Recife", "country": "Brazil", "coords": (-34.9180, -8.0476)},
    "BEL": {"city": "Belém", "country": "Brazil", "coords": (-48.4902, -1.4558)},
    "CNF": {"city": "Belo Horizonte", "country": "Brazil", "coords": (-44.0028, -19.6244)},
    "CGB": {"city": "Cuiabá", "country": "Brazil", "coords": (-56.0967, -15.5945)},
    "POA": {"city": "Porto Alegre", "country": "Brazil", "coords": (-51.1714, -30.0346)},
    "FLN": {"city": "Florianópolis", "country": "Brazil", "coords": (-48.5480, -27.5954)},
    
    # Chile
    "SCL": {"city": "Santiago", "country": "Chile", "coords": (-70.6483, -33.4489)},
    
    # Colombia
    "BOG": {"city": "Bogotá", "country": "Colombia", "coords": (-74.0721, 4.7110)},
    
    # Ecuador
    "UIO": {"city": "Quito", "country": "Ecuador", "coords": (-78.4678, -0.1807)},
    "GYE": {"city": "Guayaquil", "country": "Ecuador", "coords": (-79.8940, -2.1700)},
    "CUE": {"city": "Cuenca", "country": "Ecuador", "coords": (-79.0059, -2.8974)},
    
    # Guyana
    "GEO": {"city": "Georgetown", "country": "Guyana", "coords": (-58.2541, 6.8013)},
    
    # Paraguay
    "ASU": {"city": "Asunción", "country": "Paraguay", "coords": (-57.5759, -25.2637)},
    
    # Peru
    "LIM": {"city": "Lima", "country": "Peru", "coords": (-77.0428, -12.0464)},
    "AQP": {"city": "Arequipa", "country": "Peru", "coords": (-71.5375, -16.4090)},
    
    # Suriname
    "PBM": {"city": "Paramaribo", "country": "Suriname", "coords": (-55.1914, 5.4521)},
    
    # Uruguay
    "MVD": {"city": "Montevideo", "country": "Uruguay", "coords": (-56.1645, -34.9011)},
    
    # Venezuela
    "CCS": {"city": "Caracas", "country": "Venezuela", "coords": (-66.9036, 10.4806)}
}

In [ ]:
# define different flight paths (corrected)
routes = {
    "Direct Flight": ["BOG", "EZE"],
    "Via São Paulo": ["BOG", "GRU", "EZE"],
    "Via Lima": ["BOG", "LIM", "EZE"],
    "Via Santiago": ["BOG", "SCL", "EZE"],
    "Via Santa Cruz": ["BOG", "VVI", "EZE"],
    "Via Lima & Santiago": ["BOG", "LIM", "SCL", "EZE"],
    "Via Lima & Asunción": ["BOG", "LIM", "ASU", "EZE"],
    "Via Lima & Santa Cruz": ["BOG", "LIM", "VVI", "EZE"],
    "Via Santa Cruz & Santiago": ["BOG", "VVI", "SCL", "EZE"],
    "Via Quito & Lima": ["BOG", "UIO", "LIM", "EZE"],
    "Via Caracas & Lima": ["BOG", "CCS", "LIM", "EZE"],
    "Via Quito & Santiago": ["BOG", "UIO", "SCL", "EZE"],
    "Via Caracas & Santa Cruz": ["BOG", "CCS", "VVI", "EZE"],
    "Via Quito & Asunción": ["BOG", "UIO", "ASU", "EZE"],
    "Via Montevideo": ["BOG", "MVD", "EZE"],
    "Via Rio de Janeiro": ["BOG", "GIG", "EZE"],
    "Via Brasília & São Paulo": ["BOG", "BSB", "GRU", "EZE"],
    "Via Brasília & Santa Cruz": ["BOG", "BSB", "VVI", "EZE"],
    "Via Manaus": ["BOG", "MAO", "EZE"],
    "Via Recife": ["BOG", "REC", "EZE"],
    "Via Fortaleza": ["BOG", "FOR", "EZE"],
    "Via Belém": ["BOG", "BEL", "EZE"],
    "Via Cuiabá": ["BOG", "CGB", "EZE"],
    "Via Porto Alegre": ["BOG", "POA", "EZE"],
    "Via Florianópolis": ["BOG", "FLN", "EZE"],
    "Via Cordoba": ["BOG", "COR", "EZE"],
    "Via Guayaquil": ["BOG", "GYE", "EZE"],
    "Via La Paz": ["BOG", "LPB", "EZE"],
    "Via Arequipa": ["BOG", "AQP", "EZE"],
    "Via Cuenca": ["BOG", "CUE", "EZE"],
    "Via Rosario": ["BOG", "ROS", "EZE"],
    "Via Mendoza": ["BOG", "MDZ", "EZE"],
    "Via Mar del Plata": ["BOG", "MDQ", "EZE"],
    "Via Neuquén": ["BOG", "NQN", "EZE"]
}

In [ ]:
from shapely.geometry import Point

# filter the airport codes using a dictionary comprehension
subset_airport_codes = {
    code: data
    for code, data in airports_code_map.items()
    if code in {"EZE", "BOG", "LIM", "VVI", "SCL", "ASU", "GRU"}
}

# filter the routes using a dictionary comprehension
subset_routes = {
    route_name: airports
    for route_name, airports in routes.items()
    if all(code in subset_airport_codes.keys() for code in airports)
}

# create GeoDataFrame for airports
airports_subset_bog_to_eze = gpd.GeoDataFrame(
    {
        "City": [f"{data['city']} ({key})" for key, data in subset_airport_codes.items()],
        "Country": [data["country"] for data in subset_airport_codes.values()],
        "geometry": [Point(data["coords"]) for data in subset_airport_codes.values()]
    },
    crs="EPSG:4326",
)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from adjustText import adjust_text
from shapely.geometry import LineString


# set initial figure counter to 1
fig_count = 1

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

# get subplot for South America
fig, ax = plt.subplots(figsize=(16, 16))

# set the view for only the mainland
ax.set_xlim([-85, -30])
ax.set_ylim([-60, 15])

# turn off axes for the South America plot only
ax.axis("off")

# plot all countries with darkish grey color
south_america.plot(
    ax=ax,
    color=plt.cm.viridis(0.67),
    edgecolor="#181818",
    linewidth=0.2,
);

# plot flight paths with a specific label for each route
for route, path in subset_routes.items():
    # Set color to red for Direct Flight, else a different color
    if "Direct" in route:
        color = "red"
        style = "solid"

    else:
        color = plt.cm.inferno(1.0)
        style = "dashed"

    # Extract the coordinates for each airport code
    coords = [subset_airport_codes[code]["coords"] for code in path]
    
    # Create a LineString and plot the route
    line = LineString(coords)
    ax.plot([point[0] for point in line.coords], [point[1] for point in line.coords],
            color=color, linewidth=2, linestyle=style, alpha=0.7)

# plot airports
airports_subset_bog_to_eze.plot(ax=ax, color="red", markersize=50, edgecolor="black")

texts = []
for x, y, label in zip(
    airports_subset_bog_to_eze.geometry.x,
    airports_subset_bog_to_eze.geometry.y,
    airports_subset_bog_to_eze["City"]):
    text = ax.text(x, y, label, fontsize=8, ha="right", color="black", bbox=dict(facecolor="white", alpha=0.7))
    texts.append(text)

# Manually add the legend for "Direct Flight" and "Non-Direct Flight"
legend_elements = [
    Line2D([0], [0], color="red", lw=2, linestyle="solid", label="Direct Flight"),
    Line2D([0], [0], color=plt.cm.inferno(1.0), lw=1, linestyle="dashed", label="Non-Direct Flight")
]

# Add legend
ax.legend(handles=legend_elements, loc="upper right", fontsize=8, frameon=False)

# set title
plt.suptitle(
    f"Figure {fig_count}. Bogota to Buenos Aires Flight Paths.", y=0.0001, fontsize=10,
)

# increment fig count
fig_count += 1

# adjust text positioning to avoid overlap, expand text bounding boxes
adjust_text(
    texts,
    expand=(1.2, 2),  # expand bounding boxes by 1.2x in x and 2x in y directions
    arrowprops=dict(arrowstyle='->', color='red')  # arrows to ensure clarity
)

# adjust padding
plt.tight_layout(pad=0.5)  # Adjust the padding as needed

# display
plt.show()

In *figure 1* above we can see the various flight paths possible from *Bogota, Columbia* to *Buenos Aires, Argentina*. Clearly, any flight path that is not a *direct flight* will have a longer total distance. Should our *utility* function reward or penalize this? It would not make sense to *"reward"* excess distance, because that would be equivalent to *"waste"* distance. This can be understood in terms of **accuracy** or **efficiency**: we do not want to simply travel from point `A` to `Z` (e.g. *Bogota* to *Buenos Aires*) by any path, we want to get there by the path of *minimal waste*. So then how do we account for this *"waste distance"* because clearly we must consider it, otherwise our *utility function* would be *increasing* as the distance increases **beyond** the *"ideal distance."* The solution comes from understanding that the *"benefit"* gained from traveling is not an absolute distance but a *distance efficiency* ($E_D$):

$$
E_D = \frac{D_{\text{ideal}}}{D_{\text{actual}}}
$$

We are not *"buying"* raw distance, but instead an *"efficiency"* or *"accuracy"* of distance from our origin `A` to our target destination `Z`. Now we have the correct behavior:

- **$D_{\text{actual}} = D_{\text{ideal}}; E_D = 1$**: utility is maximized, since the denominator is just
  $M \cdot T \cdot S$, and there is no penalty from the actual distance.
- **$D_{\text{actual}} > D_{\text{ideal}}; E_D < 0$**: utility decreases as the actual distance grows larger.

We can now simplify the *utility function* even further:

$$
U(x) = \frac{E_D}{M \cdot T \cdot S}
$$

## Mathematics of Stress
The final term of the equation that we need to discuss is *stress* $S$. The *money* $M$ and *time* $T$ terms are relatively simple:

+ $M$ represents all the financial costs spent on the trip (e.g. ticket cost, baggage fees, etc ...)
+ $T$ represents all the *temporal* costs spent on the trip (e.g. flight time, layovers, traffic, delays, etc ...)

But *stress* $S$ is a bit more abstract and complex:

+ *Stress accumulates over time*, meaning that longer trips generally result in higher stress.
+ However, stress is not **linearly** dependent on time.

We *"intuitively"* feel this: a non-stop, 12-hour flight might feel more stressful than two 6-hour flights with a comfortable layover in between. Conversely, too many layovers can introduce additional stressors such as airport transfers, security checks, and unpredictability.

So how do we go about deriving the *mathematical model* of stress? Well the *"key"* insight here is to understand that *stress* is simply an example of [continuously compounding](https://en.wikipedia.org/wiki/Compound_interest) which is just a type of [exponential growth](https://en.wikipedia.org/wiki/Exponential_growth):

$$
S = S_0 e^{kT}
$$

where:  
- $S_0$ is the **baseline stress** (initial stress level).  
- $k$ is a **stress growth rate**, which could depend on factors like discomfort, unpredictability, or psychological burden.  
- $T$ is the **total time** spent on the trip.

Let's look at an example scenario that really shows what this *mathematical model* captures about *stress* $S$. For a **12-hour** direct flight:

$$
S_{\text{long-haul}} = S_0 e^{k(12)}
$$

For a trip broken into **three 4-hour segments** with **two layovers** (each reducing stress by a factor of $\alpha$, where $0 < \alpha < 1$:

1. **First Flight (4 hours)**:

$$
S_1 = S_0 e^{k(4)}
$$

2. **First Layover (partial stress reset)**:

$$
S_2 = \alpha S_1 = \alpha S_0 e^{k(4)}
$$

3. **Second Flight (4 hours)**:

$$
S_3 = S_2 e^{k(4)} = \alpha S_0 e^{k(4)} e^{k(4)} = \alpha S_0 e^{k(8)}
$$

4. **Second Layover (another stress reset)**:

$$
S_4 = \alpha S_3 = \alpha^2 S_0 e^{k(8)}
$$

5. **Final Flight (4 hours)**:

$$
S_5 = S_4 e^{k(4)} = \alpha^2 S_0 e^{k(8)} e^{k(4)} = \alpha^2 S_0 e^{k(12)}
$$

Thus, the final stress for the segmented trip is:

$$
S_{\text{segmented}} = \alpha^2 S_0 e^{k(12)}
$$

If we compare the two *travel strategies*:

1. **Long-haul flight stress**:

$$
S_{\text{long-haul}} = S_0 e^{k(12)}
$$

2. **Segmented flight stress (with layovers reducing stress by $\alpha^2$)**:

$$
S_{\text{segmented}} = \alpha^2 S_0 e^{k(12)}
$$

Since $0 < \alpha < 1$, we have:

$$
S_{\text{segmented}} < S_{\text{long-haul}}
$$

indicating that **layovers help reduce accumulated stress**, despite the total time being the same. Certainly this depends on $\alpha < 1$, otherwise we would not see such *"benefit"* from *interrupting stress growth*. To make this more clear let's look at the same example but with actual values:

- **Baseline stress level**: $S_0 = 10$ (arbitrary unit)
- **Stress growth rate**: $k = 0.1$ per hour
- **Total travel time**: $T = 12$ hours  
- **Layover stress reduction factor**: $\alpha = 0.5$ (each layover halves the accumulated stress)

First we calculate the continuous *long-haul* flight:

$$
S_{\text{long-haul}} = 10 \cdot e^{0.1 \times 12}
$$

$$
S_{\text{long-haul}} = 10 \cdot e^{1.2}
$$

$$
S_{\text{long-haul}} \approx 10 \cdot 3.32 = 33.2
$$

Now the *multi-segment* flight:

1. **First Flight (4 hours)**:

$$
S_1 = 10 \cdot e^{0.1 \times 4} = 10 \cdot e^{0.4}
$$

$$
S_1 \approx 10 \cdot 1.49 = 14.9
$$

2. **First Layover (partial stress reset)**:

$$
S_2 = 0.5 \times 14.9 = 7.45
$$

3. **Second Flight (4 hours)**:

$$
S_3 = 7.45 \cdot e^{0.1 \times 4} = 7.45 \cdot e^{0.4}
$$

$$
S_3 \approx 7.45 \cdot 1.49 = 11.1
$$

4. **Second Layover (another stress reset)**:

$$
S_4 = 0.5 \times 11.1 = 5.55
$$

5. **Final Flight (4 hours)**:

$$
S_5 = 5.55 \cdot e^{0.1 \times 4} = 5.55 \cdot e^{0.4}
$$

$$
S_5 \approx 5.55 \cdot 1.49 = 8.3
$$

The final comparison:
- **Long-haul stress**: $S_{\text{long-haul}} \approx 33.2$
- **Segmented flight stress**: $S_{\text{segmented}} \approx 8.3$

So, despite both flights covering **12 hours**, breaking it into *3 segments with 2 layovers* **reduced stress from 33.2 to 8.3**, which is a **75% reduction in final stress**! Key insights:

- A direct flight accumulates **continuous** stress exponentially.
- Layovers **reset** stress by a factor $\alpha$, preventing excessive buildup.
- Even though layovers might be inconvenient, they can significantly reduce total travel stress.

This illustrates why **ultra-long-haul flights** might feel much more exhausting compared to breaking the journey into shorter, more manageable segments.

## Growing Stress
But one aspect of the *mathematics of stress* we have not discussed is what *influences* the *growth rate* $k$? For our purposes, let us consider some *"travel examples"* in an attempt to reason about what causes *"stress"* in travel. Consider the following two scenarios:

**Scenario 1**: Empty Train
- You board a **nearly empty** train car.  
- There is **ample seating**, personal space, and no loud distractions.  
- No one is **blocking the aisles**, slowing you down, or invading your personal space.  
- The ride is **quiet**, predictable, and uninterrupted.

**Scenario 2**: Packed Subway
- You board a **rush-hour subway**, standing shoulder to shoulder with strangers.  
- Noise levels are high: **announcements, conversations, crying babies**.  
- The **air is stuffy**, and you’re forced to hold onto a metal bar with limited mobility.  
- At each stop, **more people cram in**, pushing against you.  
- The train **suddenly stops** due to delays, adding uncertainty.

There are many different ways we can *"describe"* what is the difference between the two scenarios: *noise*, *threat*, *attack*, *interruption*, *diversion*, *chaos*, *distraction*, etc ... But ultimately one word encapsulates the difference: **stimulation**. The difference between the *two scenarios* is that **scenario 1** is less stimulating. And why? There is less *stimulation* from the environment: less people, less noise, less distractions, and less interruptions. Whereas **scenario 2** is **total chaos**.

But how might we represent this *mathematically*? We simply need to account for both the *frequency* and the *amplification* of *each stimuli*:

$$
k = k_0 + \sum_{i} \sigma_i \lambda_i
$$

where:
- $k_0$ = **intrinsic stress growth rate** (baseline stress accumulation).
- $i$ = index for different types of stimuli.
- $\sigma_i$ = **stress amplification factor** for stimulus type $i$.
- $\lambda_i$ = **stimulus frequency** (events per minute, per hour, etc.).

Now we can return to the *two scenarios* above:

**Scenario 1**: Empty Train
- **Very few passengers** → low interaction frequency.  
- **Minimal announcements**.  
- **Smooth ride, few disturbances**.  

We assign reasonable values for the stress equation:  

$$
S = S_0 e^{kT}
$$

where the growth rate $k$ is influenced by **stimulus frequency**:

$$
k = \sigma_{\text{people}} \cdot \lambda_{\text{people}} + \sigma_{\text{announcements}} \cdot \lambda_{\text{announcements}} + \sigma_{\text{movement}} \cdot \lambda_{\text{movement}}
$$

Using estimated values:  

| **Stimulus**    | **Impact Factor** $\sigma$ | **Frequency** $\lambda$ (per min) |
|-----------------|-------------------|----------------|
| People moving   | $0.2$             | $1$            |
| Announcements   | $0.3$             | $0.2$          |
| Train movement  | $0.1$             | $1$            |

$$
k_{\text{empty}} = (0.2 \times 1) + (0.3 \times 0.2) + (0.1 \times 1) = 0.2 + 0.06 + 0.1 = 0.36
$$

For a **30-minute ride**, assuming $S_0 = 1$:

$$
S_{\text{empty}} = 1 \cdot e^{0.36 \times 30} \approx e^{10.8} \approx 49,402
$$  

 **Scenario 2**: Packed Subway:
 
- **Crowded conditions** → high people density.  
- **Frequent loud announcements**.  
- **Jostling, noise, and disruptions**.  

Using estimated values:  

| **Stimulus**    | **Impact Factor** $\sigma$ | **Frequency** $\lambda$ (per min) |
|------------------|------------------|----------------|
| People moving   | $0.2$         | $10$      |
| Announcements   | $0.3$         | $2$       |
| Train movement  | $0.1$         | $5$       |

$$
k_{\text{packed}} = (0.2 \times 10) + (0.3 \times 2) + (0.1 \times 5) = 2 + 0.6 + 0.5 = 3.1
$$

For the same **30-minute ride**, with $S_0 = 1$:

$$
S_{\text{packed}} = 1 \cdot e^{3.1 \times 30} \approx e^{93} \approx 1.08 \times 10^{40}
$$

The stress growth rate in a **packed subway** ($k = 3.1$) is exponentially higher than in an **empty train** ($k = 0.36$), leading to an *astronomical difference* in total stress over time. This highlights how a crowded, noisy environment rapidly compounds stress, while a quiet, low-stimulus setting keeps it minimal. Ultimately, **stimulus frequency** — the rate of encounters with people, noise, and movement — emerges as the key driver of stress accumulation.

## Price Utility
One final point needs to be considered before we actually apply the *travel utility function* in the next section, and that is to discuss the *optional* usage of *price utility* as an alternative to *total price*. Instead of summing all absolute costs:  

$$
M = P_{\text{total}} = P_{\text{ticket}} + P_{\text{fees}} + P_{\text{hotel}} + \dots
$$

we redefine price in terms of *price utility* by normalizing costs relative to wealth $W$:  

$$
M = \frac{P_{\text{total}}}{W} = \frac{P_{\text{ticket}} + P_{\text{fees}} + P_{\text{hotel}} + \dots}{W}
$$

Substituting for $M$ in the *travel utility function*:

$$
\begin{aligned}
U(x) &= \frac{D_{\text{ideal}}}{\frac{P_{\text{total}}}{W} \cdot T \cdot S \cdot D_{\text{actual}}} \\
U(x) &= \frac{D_{\text{ideal}} \cdot W}{P_{\text{total}} \cdot T \cdot S \cdot D_{\text{actual}}}
\end{aligned}
$$

This formulation acknowledges that the same monetary cost has a vastly different impact depending on the traveler’s financial situation. A $500$ expense may be trivial for one person yet burdensome for another. This *"optional"* way of calculating the $M$ value will be useful when we want to consider the *optimal travel option* for individuals with differing values of *wealth* $W$.

## Synthetic Travel Data
Now we can actually apply the *travel utility function* to the previous *travel plan* (i.e. *Bogota* -> *Buenos Aires*). But we first will need to generate some *synthetic travel data* that allows us to *simulate* what the *travel options* might look like:

In [ ]:
import numpy as np

# for flipping lat/long coordinates
def flip_coords(coord):
    return (coord[1], coord[0])

# function to calculate the great-circle distance (in km) between two points
def calculate_distance(p1, p2):
    lat1, lon1 = np.radians(p1)
    lat2, lon2 = np.radians(p2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return 6371 * c  # radius of Earth in km

def variable_flight_speed(distance_km: float) -> float:
    """Simulates a variable flight speed over the course of a flight."""
    # define distance percentages for different phases
    climb_percent = 0.1
    cruise_percent = 0.8
    descent_percent = 0.1

    # calculate distance covered in each phase
    climb_distance = distance_km * climb_percent
    cruise_distance = distance_km * cruise_percent
    descent_distance = distance_km * descent_percent

    # fixed average speeds for each phase in km/h
    climb_speed = 600  # Climb phase speed
    cruise_speed = 800  # Cruise phase speed
    descent_speed = 600  # Descent phase speed

    # calculate flight time for each phase
    climb_time = climb_distance / climb_speed
    cruise_time = cruise_distance / cruise_speed
    descent_time = descent_distance / descent_speed

    # sum total flight time
    total_time = climb_time + cruise_time + descent_time

    return total_time

# define constants
D_ideal = calculate_distance(
    flip_coords(airports_code_map["BOG"]["coords"]),
    flip_coords(airports_code_map["EZE"]["coords"])
)
BASE_COST_PER_1000KM = 150

# travel class impact on stress growth rate (lower k means less stress)
class_k_modifier = {
    "Economy": 1.0,
    "Business": 0.7,
    "First Class": 0.3
}

# day of week impact (weekends are less stressful for travel)
day_k_modifier = {
    "Mon": 1.2, "Tue": 1.1, "Wed": 1.1, "Thu": 1.0,
    "Fri": 1.3, "Sat": 0.9, "Sun": 0.8
}

# month impact on stress (holidays and peak travel months like December or July increase stress)
month_k_modifier = {
    "Jan": 1.1, "Feb": 1.0, "Mar": 1.0, "Apr": 1.0, "May": 1.0, "Jun": 1.1,
    "Jul": 1.3, "Aug": 1.0, "Sep": 1.0, "Oct": 1.0, "Nov": 1.1, "Dec": 1.3
}

# time of day impact (red-eye flights = less stress)
time_k_modifier = {
    "Morning": 1.2, "Afternoon": 1.1, "Evening": 1.0, "Red-Eye": 0.7
}

# weather conditions and their impact on stress
weather_k_modifier = {
    "Clear": 0.8,   # ideal conditions
    "Cloudy": 1.0,  # neutral
    "Rainy": 1.2,   # increased stress
    "Stormy": 1.5,  # very stressful
    "Windy": 1.3    # moderately stressful, especially during takeoff and landing
}

# lounge access types and stress modifiers
lounge_access = {
    "None": 1.0,        # no reduction in stress
    "Basic": 0.9,       # mild reduction in stress
    "Premium": 0.7,     # moderate reduction in stress
    "VIP": 0.5,         # significant reduction in stress
    "Private Room": 0.2 # near-zero stress
}

# class multipliers (how much more expensive each class is compared to Economy)
class_price_multiplier = {
    "Economy": 1.0,
    "Business": 2.0,
    "First Class": 4.0
}

# lounge access cost multipliers
lounge_price_multiplier = {
    "None": 1.0,
    "Basic": 1.1,
    "Premium": 1.3,
    "VIP": 1.6,
    "Private Room": 2.0
}

# stress tier definitions (S_0)
stress_tiers = {
    "low": 1,
    "mild": 3,
    "moderate": 5,
    "high": 8,
    "extreme": 10
}

# stress growth rate modifiers for each airline
airline_k_modifier = {
    "Avianca": 0.9,
    "LATAM": 1.0,
    "Aerolineas Argentinas": 1.1,
    "Gol": 1.2,
    "Sky Airline": 1.3,
    "Viva Air": 1.4,
    "Jetsmart": 1.5,
    "BoA": 1.1,
    "Amaszonas": 1.3,
    "Paranair": 1.4,
    "Copa": 1.0,
    "Azul": 1.2
}

# price multipliers for each airline
airline_price_modifier = {
    "Avianca": 1.1,
    "LATAM": 1.2,
    "Aerolineas Argentinas": 1.3,
    "Gol": 1.0,
    "Sky Airline": 0.9,
    "Viva Air": 0.8,
    "Jetsmart": 0.85,
    "BoA": 1.0,
    "Amaszonas": 1.05,
    "Paranair": 1.2,
    "Copa": 1.15,
    "Azul": 1.1
}

In [ ]:
# airlines that fly each route
route_airlines = {
    "Direct Flight": ["Avianca", "LATAM", "Aerolineas Argentinas"],
    "Via São Paulo": ["LATAM", "Gol", "Avianca + Gol", "Azul"],
    "Via Lima": ["LATAM", "Sky Airline", "Viva Air + LATAM"],
    "Via Santiago": ["LATAM", "Sky Airline", "Jetsmart"],
    "Via Santa Cruz": ["BoA", "Amaszonas", "LATAM"],
    "Via Lima & Santiago": ["LATAM", "Sky Airline + LATAM"],
    "Via Lima & Asunción": ["LATAM + Paranair", "Copa + Paranair"],
    "Via Lima & Santa Cruz": ["LATAM + BoA", "Sky Airline + BoA"],
    "Via Santa Cruz & Santiago": ["BoA + LATAM", "Amaszonas + Sky Airline"],
    "Via Quito & Lima": ["LATAM", "Avianca + LATAM"],
    "Via Caracas & Lima": ["Avianca", "LATAM"],
    "Via Quito & Santiago": ["LATAM", "Sky Airline"],
    "Via Caracas & Santa Cruz": ["Avianca + LATAM", "Amaszonas"],
    "Via Quito & Asunción": ["Avianca", "LATAM"],
    "Via Montevideo": ["Avianca", "LATAM", "Aerolineas Argentinas"],
    "Via Rio de Janeiro": ["Avianca", "LATAM", "Gol"],
    "Via Brasília & São Paulo": ["Avianca + Gol", "LATAM", "Gol"],
    "Via Brasília & Santa Cruz": ["LATAM", "Amaszonas"],
    "Via Manaus": ["Avianca", "LATAM", "Gol"],
    "Via Recife": ["Avianca", "LATAM", "Gol"],
    "Via Fortaleza": ["Avianca", "LATAM", "Gol"],
    "Via Belém": ["Avianca", "LATAM", "Gol"],
    "Via Cuiabá": ["Avianca", "LATAM", "Gol"],
    "Via Porto Alegre": ["Avianca", "LATAM", "Gol"],
    "Via Florianópolis": ["Avianca", "LATAM", "Gol"],
    "Via Cordoba": ["Avianca", "Aerolineas Argentinas", "LATAM"],
    "Via Guayaquil": ["Avianca", "LATAM"],
    "Via La Paz": ["Avianca", "LATAM"],
    "Via Arequipa": ["Avianca", "LATAM", "Sky Airline"],
    "Via Cuenca": ["Avianca", "LATAM"],
    "Via Rosario": ["Aerolineas Argentinas", "LATAM"],
    "Via Mendoza": ["Aerolineas Argentinas", "LATAM"],
    "Via Mar del Plata": ["Aerolineas Argentinas", "LATAM"],
    "Via Neuquén": ["Aerolineas Argentinas", "LATAM"]
}

In [ ]:
import itertools
import numpy as np
from typing import List, Dict, Generator, Any, Optional, Tuple

# set seed for reproducibility
np.random.seed(42)

def generate_layover_times(
    size: int = 1000, mu: float = 1.0, sigma: float = 0.75, min_layover: float = 1.0, max_layover: float = 10.0
) -> np.ndarray:
    """Generates random layover times based on a log-normal distribution."""
    # generate log-normal distributed values
    layover_times = np.random.lognormal(mean=mu, sigma=sigma, size=size)
    
    # clip values to the specified range (between min_layover and max_layover)
    layover_times = np.clip(layover_times, min_layover, max_layover)
    
    # include 0 as a valid layover time, representing a direct flight with no layover
    layover_times = np.append(layover_times, 0)
    
    return layover_times

def get_max_layovers(routes: Dict[str, Any]) -> int:
    """Finds the maximum number of layovers in the given routes dictionary."""
    # set initial count
    max_layovers_count = 0

    # loop over routes
    for path in routes.values():
        # calculate layover number
        layovers = len(path) - 2

        # update max
        max_layovers_count = max(max_layovers_count, layovers)
    
    return max_layovers_count

def generate_layover_combinations(
    layover_times: List[float], 
    lounge_types: List[str], 
    max_layovers: int
) -> Generator[List[Tuple[float, str]], None, None]:
    """Generate all possible valid layover combinations."""
    # direct flight with no layovers
    yield [(0, "None")]
 
    # get combos
    layover_combinations = list(itertools.product(layover_times, lounge_types))
    
    # generate combinations for each number of layovers
    for num_layovers in range(1, max_layovers + 1):
        for combination in itertools.product(layover_combinations, repeat=num_layovers):
            yield list(combination)

def generate_all_combinations(
    routes: Dict[str, List[str]],
    airlines: List[str],
    travel_class: List[str],
    day_of_week: List[str],
    month: List[str],
    time_of_day: List[str],
    weather: List[str],
    layover_times: List[float],
    lounge_types: List[str]
) -> Generator[Dict[str, Any], None, None]:
    """Generate all possible combinations of flight parameters (exhaustive)."""
    # generate flight parameters combinations
    for route, airline, t_class, day, m, time, w, layover in itertools.product(
        routes.keys(),
        airlines,
        travel_class,
        day_of_week,
        month,
        time_of_day,
        weather,
        generate_layover_combinations(layover_times, lounge_types, get_max_layovers(routes))
    ):
        # calculate the number of layovers for the route
        num_layovers = len(routes[route]) - 2

        # check if the airline is allowed for the current route
        if airline not in route_airlines.get(route, []):
            continue  # Skip this combination if the airline is not allowed for the route

        # check if it's a direct flight, layover must be [(0, 'None')]
        if 'Direct Flight' in route and layover != [(0, 'None')]:
            continue  # Skip direct flights with any layovers

        # check if it's a flight with layovers, the number of layovers should match
        if 'Via' in route and len(layover) != num_layovers:
            continue  # Skip combinations where the layover count doesn't match the route

        # extract layover times and lounges from the layover combination (which is a list of tuples)
        layover_times_comb = [time for time, _ in layover]
        lounges = [lounge for _, lounge in layover]

        # yield the valid combination of parameters
        yield {
            "route": route,
            "airline": airline,
            "travel_class": t_class,
            "day_of_week": day,
            "month": m,
            "time_of_day": time,
            "weather": w,
            "layover_times": layover_times_comb,
            "lounges": lounges
        }

def generate_random_combinations(
    routes: Dict[str, List[str]],
    airlines: List[str],
    travel_class: List[str],
    day_of_week: List[str],
    month: List[str],
    time_of_day: List[str],
    weather: List[str],
    layover_times: List[float],
    lounge_types: List[str],
    random_samples: int
) -> Generator[Dict[str, Any], None, None]:
    """Generate multiple random samples of flight parameters."""
    # begin loop to generate random data
    for _ in range(random_samples):
        route = np.random.choice(list(routes.keys()))
        airline = np.random.choice(airlines)
        t_class = np.random.choice(travel_class)
        day = np.random.choice(day_of_week)
        m = np.random.choice(month)
        time = np.random.choice(time_of_day)
        w = np.random.choice(weather)

        # determine number of layovers required
        layover_count = len(routes[route]) - 2

        # generate layover times (based on number of layovers in the route)        
        if layover_count > 0:
            lay_times = np.random.choice(layover_times, size=layover_count)
        else:
            # no layovers for direct flights
            lay_times = [0]

        # generate lounge type (ensure it matches the number of layovers)
        if layover_count == 0:
            # no layovers means no lounge
            lounges = ["None"]
        else:
            # optional lounge access for each layover
            lounges = np.random.choice(lounge_types, size=layover_count)

        yield {
            "route": route,
            "airline": airline,
            "travel_class": t_class,
            "day_of_week": day,
            "month": m,
            "time_of_day": time,
            "weather": w,
            "layover_times": lay_times,
            "lounges": lounges
        }

def generate_flight_parameters(
    routes: Dict[str, List[str]],
    airlines: Optional[List[str]] = None,
    travel_class: Optional[List[str]] = None,
    day_of_week: Optional[List[str]] = None,
    month: Optional[List[str]] = None,
    time_of_day: Optional[List[str]] = None,
    weather: Optional[List[str]] = None,
    layover_times: Optional[List[float]] = None,
    lounge_types: Optional[List[str]] = None,
    random_samples: Optional[int] = None
) -> Generator[Dict[str, Any], None, None]:
    """Generate flight parameters either randomly or exhaustively."""
    # default values for parameters if they are not passed
    airlines = airlines or list(airline_k_modifier.keys())
    travel_class = travel_class or list(class_k_modifier.keys())
    day_of_week = day_of_week or list(day_k_modifier.keys())
    month = month or list(month_k_modifier.keys())
    time_of_day = time_of_day or list(time_k_modifier.keys())
    weather = weather or list(weather_k_modifier.keys())
    layover_times = layover_times or generate_layover_times(10)
    lounge_types = lounge_types or list(lounge_access.keys())

    # prepare a dictionary with the parameters to pass
    params_dict = {
        'routes': routes,
        'airlines': airlines,
        'travel_class': travel_class,
        'day_of_week': day_of_week,
        'month': month,
        'time_of_day': time_of_day,
        'weather': weather,
        'layover_times': layover_times,
        'lounge_types': lounge_types
    }

    # set default generator function
    params_generator = generate_all_combinations

    # check for random_samples and update accordingly
    if random_samples:
        params_dict['random_samples'] = random_samples
        params_generator = generate_random_combinations

    # use kwargs unpacking to pass parameters to the generator
    yield from params_generator(**params_dict)

In [ ]:
import pandas as pd
from typing import Iterable

def generate_flight_utility_data(
    flight_params: Iterable[Dict[str, Any]],
    base_stress: Optional[str] = None,
    wealth: Optional[float] = None,
    utility_scale: float = 1.0
) -> pd.DataFrame:
    """
    Generates synthetic flight utililty data based on various travel factors such as route, class, 
    time of travel, weather conditions, and stress modifiers.
    
    Args:
        flight_params (Iterable[Dict[str, Any]]):
            An iterable (e.g., list) containing pre-generated flight parameters.
        base_stress (Optional[str], optional):
            If set, the stress level will be based on the given tier ('low', 'moderate', 'high', 'extreme');
            If None (default), the stress level will be randomized.
        wealth (Optional[float], optional):
            If provided, an additional price utility metric (`U_p = M / W`) is calculated.
        utility_scale (float):
            A scaling factor applied to the final travel utility (`U_t`).
            Defaults to 1.0 (no scaling).
    
    Returns:
        pd.DataFrame: A DataFrame containing the generated flight data, including:
            - Route
            - Airline
            - Class
            - Day
            - Month
            - Time of Day
            - Weather
            - D_actual (Total Distance)
            - T_flight (Total Flight Time)
            - T_layover (Total Layover Time)
            - M (Cost $)
            - Lounges (Airport Lounges Purchased)
            - W (Wealth, if provided)
            - U_p (Price Utility, if W is provided)
            - S_0 (Base Stress)
            - S_flight (Flight Stress)
            - S_layover (Layover Stress)
            - S_total (Total Stress)
            - U_t (Scaled Travel Utility)
    """
    # temp travel option data storage
    data = []

    # process each set of flight parameters
    for flight_param in flight_params:
        route = flight_param["route"]
        airlines = flight_param["airline"]
        travel_class = flight_param["travel_class"]
        day_of_week = flight_param["day_of_week"]
        month = flight_param["month"]
        time_of_day = flight_param["time_of_day"]
        weather = flight_param["weather"]
        layover_times = flight_param["layover_times"]
        lounge_types = flight_param["lounges"]

        # get city coordinates for the route
        airport_codes = routes[route]
        distance = 0
        flight_time_per_leg = []

        # calculate total distance and flight times
        for i in range(len(airport_codes) - 1):
            code1, code2 = airport_codes[i], airport_codes[i + 1]
            leg_distance = calculate_distance(
                flip_coords(airports_code_map[code1]["coords"]),
                flip_coords(airports_code_map[code2]["coords"])
            )
            flight_time_per_leg.append(variable_flight_speed(leg_distance))
            distance += leg_distance

        # total layover time
        T_layover = sum(layover_times)  # total layover time

        # handle composite airlines by splitting and averaging
        airline_parts = airlines.split(" + ")
        airline_k = np.mean([airline_k_modifier[airline] for airline in airline_parts])
        airline_price = np.mean([airline_price_modifier[airline] for airline in airline_parts])

        # compute stress growth rate modifiers
        k_flight = sum((
            class_k_modifier[travel_class], 
            day_k_modifier[day_of_week],
            month_k_modifier[month], 
            time_k_modifier[time_of_day],
            weather_k_modifier[weather], 
            airline_k
        ))

        k_layover = sum((
            day_k_modifier[day_of_week], 
            month_k_modifier[month], 
            time_k_modifier[time_of_day]
        ))

        # set stress tier (or randomize)
        initial_stress_lvl = np.random.choice(list(stress_tiers.keys())) if not base_stress else base_stress

        # get initial stress level
        S_0 = stress_tiers[initial_stress_lvl]

        # compute flight stress
        S_flight = sum(S_0 * np.exp(k_flight * t) for t in flight_time_per_leg)

        # compute layover(s) stress
        S_layover = 0
        for i, layover_time in enumerate(layover_times):
            # get type of lounge for the i-th layover
            lounge_type = lounge_types[i]

            # get stress modifier
            lounge_modifier = lounge_access[lounge_type]

            # modify base layover stress rate
            k_layover_adjusted = k_layover * lounge_modifier

            # update sum of all layover stress
            S_layover += S_0 * np.exp(k_layover_adjusted * layover_time)


        # total them
        S_total = S_flight + S_layover

        # get base cost
        base_cost = (distance / 1000) * BASE_COST_PER_1000KM

        # get lounge costs
        lounge_price_modifiers = [lounge_price_multiplier[lounge] for lounge in lounge_types]
        total_lounge_modifier = np.mean(lounge_price_modifiers)  # averaging across all lounges used

        # get total cost
        M = base_cost * class_price_multiplier[travel_class] * total_lounge_modifier * airline_price

        # compute travel utility
        U_t = D_ideal / (M * (sum(flight_time_per_leg) + T_layover) * S_total * distance)

        # scale travel utility
        U_t *= utility_scale

         # Create row dictionary
        row = {
            "Route": route, "Airline": airlines, "Class": travel_class, "Day": day_of_week,
            "Month": month, "Time of Day": time_of_day, "Weather": weather, "D_actual": distance,
            "T_flight": sum(flight_time_per_leg), "T_layover": T_layover, "M ($)": M, "Lounges": " + ".join(lounge_types),
            "S_0": initial_stress_lvl, "S_flight": S_flight, "S_layover": S_layover, "S_total": S_total, "U_t": U_t
        }

        # If wealth is given, compute U_p and scale U_t
        if wealth is not None:
            row["W"] = wealth
            row["U_p"] = M / wealth
            row["U_t"] *= wealth

        data.append(row)

    # define column order
    columns = [
        "Route", "Airline", "Class", "Day", "Month", "Time of Day", "Weather",
        "D_actual", "T_flight", "T_layover", "M ($)", "Lounges"
    ]

    # insert right after "M ($)"
    if wealth is not None:
        columns.extend(["W", "U_p"])

    # default
    columns.extend(["S_0", "S_flight", "S_layover", "S_total", "U_t"])

    # create dataframe
    flight_df = pd.DataFrame(data, columns=columns)

    # rename the inherent index to the flight 'id'
    flight_df.index.name = 'id'

    # finished product
    return flight_df

In [ ]:
# get ranndom flight params
sample_flight_params = list(generate_flight_parameters(subset_routes, random_samples=100))

# get some sample flight utility data
sample_flights_df = generate_flight_utility_data(sample_flight_params, base_stress="low", utility_scale=1e16)

# get the highest U_t scored flights for each route
sample_best_routes = sample_flights_df.loc[sample_flights_df.groupby('Route')['U_t'].idxmax()]

# sort by U_t greatest -> least
sorted_best_routes = sample_best_routes.sort_values(by='U_t', ascending=False)

# split the DataFrame into two halves
halfway = len(sorted_best_routes.columns) // 2
first_half = sorted_best_routes.iloc[:, :halfway]
second_half = sorted_best_routes.iloc[:, halfway:]

# Print the two halves separately
print(first_half.round(2).to_string())
print("\n")
print(second_half.round(2).to_string())

 Here is a breakdown of what each column in the above *sample data* represents:  

- **`Route`**: The specific flight route taken, including possible layovers or connections.
  <br><br>
- **`Airline`**: The airline or airlines providing the travel service. If multiple airlines are used, they are separated by a
  `+` symbol.
  <br><br>
- **`Class`**: The travel class for the majority of the trip, which influences stress and comfort (e.g., Economy, Business,
  First Class).
  <br><br>
- **`Day`**: The day of the week on which the journey begins (e.g., Mon, Tue, Wed).
  <br><br>
- **`Month`**: The month in which the journey begins (e.g., Jan, Feb, Mar).
  <br><br>
- **`Time of Day`**: The approximate time of day when the flight begins (e.g., Morning, Afternoon, Evening, Red-Eye).
  <br><br>
- **`Weather`**: Weather conditions during takeoff or landing, which can influence stress (e.g., Clear, Cloudy, Stormy,
  Windy).
  <br><br>
- **`D_actual`**: The actual distance covered by the route in miles, which may be greater than the ideal distance due to
  layovers or deviations.
  <br><br>
- **`T_flight`**: Total inflight time in hours.
  <br><br> 
- **`T_layover`**: Total layover time in hours.
  <br><br>
- **`M ($)`**: Total monetary cost of the journey in U.S. dollars.
  <br><br>
- **`Lounges`**: Available lounges or relaxation areas during layovers, which affect stress (e.g., VIP, Basic, Premium,
  Private Room).
  <br><br>
- **`S_0`**: Initial stress level, which starts at a moderate value for all options but can escalate based on various factors.
  <br><br>
- **`S_flight`**: Stress accumulated during the flight, influenced by factors such as class, flight duration, and weather.
  <br><br>  
- **`S_layover`**: Stress accumulated during layovers, influenced by factors such as available lounges, duration, and crowd
  density.
  <br><br>
- **`S_total`**: Total stress for the entire journey, calculated as the sum of `S_flight` and `S_layover`.
  <br><br>
- **`U_t`**: The calculated travel utility score for the route, where higher values represent better travel options.  

The $U_t$ column is calculated using the travel utility function:

$$
U_t = \frac{D_\text{ideal}}{M \times (T_\text{flight} + T_\text{layover}) \times S_\text{total} \times D_\text{actual}}
$$

Higher utility values indicate more favorable travel options considering distance, cost, stress, and time. The $D_{\text{actual}}$ values vary depending on different stopovers and connecting routes, which can increase total distance traveled.

Now let's just assume that the above *sample data* represents the *flight options* available to us, and let's plot the routes, and color them by their $U_t$ value (i.e. a *heatmap*):

In [ ]:
from matplotlib.colors import PowerNorm

# get subplot for South America
fig, ax = plt.subplots(figsize=(22, 22))

# set the view for only the mainland
ax.set_xlim([-85, -30])
ax.set_ylim([-60, 15])

# turn off axes for the South America plot only
ax.axis("off")

# plot all countries with darkish grey color
south_america.plot(
    ax=ax,
    color=plt.cm.viridis(0.67),
    edgecolor="#181818",
    linewidth=0.2,
);

# Normalize U_t values for colormap
U_t_values = np.log1p(sorted_best_routes["U_t"])
vmin, vmax = np.percentile(U_t_values, [5, 95])    # focus on the middle 90% of the data
norm = PowerNorm(gamma=0.5, vmin=vmin, vmax=vmax)  # nonlinear scaling for better contrast
cmap = plt.cm.plasma                               # improved colormap for perceptual contrast

# jitter amount for separating overlapping routes
jitter_strength = 0.3                              # increase this to separate more

# Plot flight paths colored by U_t
for idx, row in sorted_best_routes.iterrows():
    route = row["Route"]
    U_t = row["U_t"]    
    
    # extract the coordinates for each airport code in the route
    coords = [subset_airport_codes[code]["coords"] for code in routes[route]]
    
    # apply jitter to each coordinate
    jittered_coords = []
    for lon, lat in coords:
        jitter_lon = lon + np.random.uniform(-jitter_strength, jitter_strength)
        jitter_lat = lat + np.random.uniform(-jitter_strength, jitter_strength)
        jittered_coords.append((jitter_lon, jitter_lat))
    
    # create a LineString and plot the route
    line = LineString(jittered_coords)
    color = cmap(norm(np.log1p(U_t)))
    ax.plot(
        [point[0] for point in line.coords], 
        [point[1] for point in line.coords],
        color=color, linewidth=1.5,
    )

# plot airports
airports_subset_bog_to_eze.plot(ax=ax, color="red", markersize=50, edgecolor="black")

texts = []
for x, y, label in zip(
    airports_subset_bog_to_eze.geometry.x,
    airports_subset_bog_to_eze.geometry.y,
    airports_subset_bog_to_eze["City"]):
    text = ax.text(x, y, label, fontsize=8, ha="right", color="black", bbox=dict(facecolor="white", alpha=0.7))
    texts.append(text)

# adjust text positioning to avoid overlap, expand text bounding boxes
adjust_text(
    texts,
    expand=(1.2, 2),
    arrowprops=dict(arrowstyle='->', color='red')
)

# add colorbar to indicate U_t value mapping
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', fraction=0.02, pad=0.04)
cbar.set_label('Travel Utility ($U_t$)', fontsize=10, color="white")
cbar.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='white')

# set title
plt.suptitle(
    f"Figure {fig_count}. Bogota to Buenos Aires Flight Paths Colored by $U_t$.", y=0.07, fontsize=10,
)

# increment fig count
fig_count += 1

# display the plot
plt.show()